# Number Recognition with the MNIST dataset

"""
This code sets up an AI using the MNIST dataset to recognize numbers.
"""

## Importing the necessary libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn

from keras.datasets import mnist

## Loading MNIST dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Data Preprocessing

In [ ]:
print('Before normalization : Min={}, max={}'.format(x_train.min(),x_train.max()))

xmax=x_train.max()
x_train = x_train / xmax
x_test  = x_test  / xmax

print('After normalization  : Min={}, max={}'.format(x_train.min(),x_train.max()))

## Model Definition

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

## Model Compilation

In [ ]:
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Model Training

In [ ]:
history = model.fit(x_train, y_train, batch_size=512, epochs=15, validation_data=(x_test, y_test))

## Evaluate the model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]:4.4f}')

Plot history

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')

plt.show()

Confusion Matrix

In [ ]:
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=-1)
confusion_matrix = tf.math.confusion_matrix(labels=y_test, predictions=predicted_classes)
print(confusion_matrix)

plt.figure(figsize = (10,7))
sn.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

plt.show()

Plot some errors

In [ ]:
y_sigmoid = model.predict(x_test, verbose=1)
y_pred = np.argmax(y_sigmoid, axis=-1)

errors=[ i for i in range(len(x_test)) if y_pred[i]!=y_test[i] ]
errors=errors[:min(24,len(errors))]

plt.figure(figsize=(10,10))

for i in range(len(errors)):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[errors[i]], cmap=plt.cm.binary)
    plt.xlabel(f'True={y_test[errors[i]]} Pred={y_pred[errors[i]]}')
    
plt.show()